In [15]:
import torch
from torch.nn.functional import one_hot
import torch_explain as te
from torch_explain.nn.functional import l1_loss
from torch_explain.logic.nn import psi
from torch_explain.logic.metrics import test_explanation , complexity


x0 = torch.zeros((4, 100))
x_train = torch.tensor([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
], dtype=torch.float)
# x_train = torch.cat([x_train, x0], dim=1)
y_train = torch.tensor([1, 0, 0, 1], dtype=torch.long)
print(x_train)
print(y_train)

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])
tensor([1, 0, 0, 1])


In [16]:
layers = [
    te.nn.EntropyLinear(x_train.shape[1], 10, n_classes=2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(10, 4),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(4, 1),
]
model = torch.nn.Sequential(*layers)

In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_form = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(1001):
    optimizer.zero_grad()
    y_pred = model(x_train).squeeze(-1)
    loss = loss_form(y_pred, y_train)
    print(y_pred, y_train)
    loss = loss_form(y_pred, y_train) + 0.00001 * te.nn.functional.entropy_logic_loss(model)
    loss.backward()
    optimizer.step()

tensor([[ -5.4080,   3.0151, -13.4801, -14.7788],
        [  3.7153,  -4.6370, -19.3326, -20.9019],
        [  2.6912,  -5.8392, -19.2713, -20.8804],
        [ -6.6231,   1.9071, -25.1238, -27.0036]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[ -5.3797,   2.9194, -13.8586, -15.1239],
        [  1.1840,  -4.9937, -19.8524, -21.5490],
        [  2.8178,  -3.4800, -19.0458, -19.4811],
        [ -3.6459,  -0.0882, -25.0395, -25.9062]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[ -5.3456,   2.9128, -14.1406, -15.3652],
        [  3.2405,  -4.8168, -20.2912, -22.0172],
        [  2.5964,  -5.5126, -18.1041, -18.5996],
        [ -5.9920,   1.5699, -24.2547, -25.2515]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[ -5.3322,   2.8338, -14.4094, -15.5937],
        [  3.7408,  -3.6796, -20.7281, -22.4825],
        [  1.6219,  -6.1067, -17.3850, -17.9728],
        [ -5.6729,   0.7175, -23.7037, -24.8616]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])


In [20]:
from torch_explain.logic.nn import entropy
from torch.nn.functional import one_hot
y1h = one_hot(y_train)

print(x_train.shape, y1h.shape)

print(y1h)
explanation, _ = entropy.explain_class(model, x_train, y1h, x_train, y1h, target_class=1)

print(model(x_train[0]))

print(explanation)

torch.Size([4, 2]) torch.Size([4, 2])
tensor([[0, 1],
        [1, 0],
        [1, 0],
        [0, 1]])
tensor([[[ -8.7202],
         [  4.0435],
         [-24.8771],
         [-25.7022]]], grad_fn=<AddBackward0>)
(feature0000000000 & feature0000000001) | (~feature0000000000 & ~feature0000000001)


In [5]:
from torch_explain.logic.metrics import test_explanation, complexity
accuracy, preds = test_explanation(explanation, x_train, y1h, target_class=1)
explanation_complexity = complexity(explanation)

print(explanation_complexity, accuracy, preds)

4 1.0 [ True False False  True]
